In [79]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [80]:
# !pip install --upgrade torchtext
# !pip install --upgrade torch

from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence
from torch import optim
import torch.nn as nn

import pandas as pd
from tqdm import tqdm

In [81]:
torch.cuda.is_available()

True

In [82]:
# from google.colab import drive
# drive.mount("/content/gdrive")


pd.set_option("display.max_colwidth", -1)
# path = "/content/gdrive/My Drive/data/amazon/amazon-reviews-max50-37k.csv"
path = "data/amazon-reviews-max50_half.csv"
df = pd.read_csv(path)

<ipython-input-82-b3cccb854e4a>:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


In [83]:
df.head()

,source,target
0,"Since putting our dog on the Wellness dog food, we don't have any more diarrhea or upset tummy issues.",So happy with Wellness
1,"I was not a huge granola fan until a friend turned me onto Dr FLynn's creation. It's not only healthier than the rest, but it tastes so much better! The best of both worlds. Can't get enough especially since it's organic! Thank you Dr Flynn!!!",Best Granola on the Planet!!!
2,OMG I cannot even begin to descibe how delicious these bars are! It is like an almond joy without the chocolate! YUM!!!,YUM!
3,The flavor of the salt is very nice ! Purchased it upon reading about it's many health benefits. The price is very reasonable as well. In local health food store it costs twice as much.,Very Happy With This Product
4,"Originally found these in a candy store in Tarrytown,NY,<br />then was delighted to find them on Amazon.<br />They are gluten free and soothe my often-irritated throat.<br />I love them and love purchasing them at Amazon with the Special Savings Prog.",Honey Candies


In [84]:
df.describe()

,source,target
count,123631,123631
unique,94250,69813
top,"Best one if you are looking for pure St John Wort Tea, all the other St John tea have other items in them hence you effects like high blood pressure get this instead it is better then a regular anti depressant and works right away",Delicious
freq,18,724


In [85]:
len(df)

123631

In [86]:
from torchtext.data import Field, TabularDataset

TEXT = Field(use_vocab=True, init_token="<SOS>", eos_token="<EOS>",
             fix_length=25, tokenize="spacy",
             include_lengths=True, batch_first=True, lower=True,
             is_target=False
             )

fields = [("source", TEXT),
          ("target", TEXT)]

data = TabularDataset(path, "CSV", fields, skip_header=True)

/home/liefe/.virtualenvs/s2s/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/liefe/.virtualenvs/s2s/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/home/liefe/.virtualenvs/s2s/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the mos

In [87]:
train_data, test_data = data.split(.70)
train_data, val_data = train_data.split(.90)

In [88]:
len(train_data), len(val_data), len(test_data)

(77888, 8654, 37089)

Build
vocab

In [89]:
# TEXT.build_vocab(train_data, vectors="fasttext.en.300d")
# TEXT.build_vocab(train_data, vectors="fasttext.en.300d")
# TEXT.build_vocab(train_data, vectors="glove.6B.100d", max_size=10000, min_freq=3)
TEXT.build_vocab(train_data, max_size=10000, min_freq=3)

#  Next time max_size=10000, min_freq=3 ?


In [90]:
print(f"Unique tokens in source/target vocabulary: {len(TEXT.vocab)}")

Unique tokens in source/target vocabulary: 10004


In [91]:
test_data[0].__dict__.keys()

dict_keys(['source', 'target'])

In [92]:
train_data[0].source, train_data[0].target

(['love',
  'the',
  'ease',
  'of',
  'having',
  'green',
  'natural',
  'tea',
  'anywhear',
  'any',
  'time',
  '.',
  'tastes',
  'great',
  'i',
  'love',
  'this',
  'stash',
  'green',
  'tea'],
 ['great', 'product'])

In [93]:
TEXT.vocab.freqs["the"], TEXT.vocab.freqs["a"], TEXT.vocab.freqs["shit"]

(98130, 67549, 4)

In [94]:
TEXT.vocab.stoi["the"], TEXT.vocab.itos[5]

(5, 'the')

In [95]:
import pprint

batch = ["Eric needs to leave".split()]
pp = pprint.PrettyPrinter(indent=4)
pad = TEXT.pad(batch)
print(pad)

([['<SOS>', 'Eric', 'needs', 'to', 'leave', '<EOS>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']], [6])


In [96]:
import random

i = random.randrange(0, len(train_data))
print(i)
source = train_data[i].source
TEXT.pad([source])

70735


([['<SOS>',
   'great',
   'lollipops',
   '...',
   'taste',
   'exactlyl',
   'like',
   'watermelon',
   '!',
   ' ',
   'used',
   'them',
   'for',
   'centerpieces',
   'for',
   'my',
   'sons',
   'barmitzvah',
   'and',
   'the',
   'kids',
   'loved',
   'them',
   '!',
   '<EOS>']],
 [25])

In [97]:
train_data[0].source

['love',
 'the',
 'ease',
 'of',
 'having',
 'green',
 'natural',
 'tea',
 'anywhear',
 'any',
 'time',
 '.',
 'tastes',
 'great',
 'i',
 'love',
 'this',
 'stash',
 'green',
 'tea']

In [98]:
train_data[0].target

['great', 'product']

In [99]:
import random

i = random.randrange(0, len(train_data))
print(i)
source = train_data[i].source
TEXT.process([source])

60524


(tensor([[   2,   13,  246,  583, 1011,   95,   20,   18,  188,  105, 1607,   24,
            10,   58,  506,   11,    6,  452,   31,   14,  136,   10, 4014,  506,
             3]]),
 tensor([25]))

Define
iterator / batcher / loader
here
note
that
this is

Model


In [100]:
class Encoder(nn.Module):
    def __init__(self, config):
        super(Encoder, self).__init__()
        self.source_dim = config["source_dim"]  # vocab size/number of classes
        self.embedding_dim = config["embedding_dim"]
        self.hidden_dim = config["hidden_dim"]
        self.layers = config.get("layers", 1)
        self.bidirectional_encoder = config.get("bidirectional_encoder", False)

        # Layers
        self.embedding = nn.Embedding(self.source_dim, self.embedding_dim)
        self.dropout = nn.Dropout(config.get("dropout", 0.0))
        self.gru = nn.GRU(self.embedding_dim, self.hidden_dim, dropout=config.get("dropout", 0.0),
                          num_layers=self.layers, bias=True, batch_first=True, bidirectional=self.bidirectional_encoder)
        # self.gru = nn.GRU(64, 64, dropout=0, num_layers=1, bias=True, batch_first=True, bidirectional=False).to(device)

    def forward(self, source, source_lens=None, pack_padded=False):

        """
        :param source: [B, T]
        :param source_lengths: [T]
        :param pack_padded: bool
        :return: output: [B, T, H], hidden: Union([1, B, H] or [2, B, H])
        """
        # print("encoder forward")
        # print(input)

        # Embed source index sequences [B, T] > [B, T, E]
        embedded = self.embedding(source)

        # print(embedded)

        # Apply dropout
        embedded = self.dropout(embedded)

        # Pack if padded sequences and lengths given

        if pack_padded:

            embedded = pack_padded_sequence(embedded, source_lens, batch_first=True, enforce_sorted=False)

        # Map embedded source [B, T, E] to Union([B, T, H] or [B, T, 2*H], Union([1, B, H] or [2, B, H])
        output, hidden = self.gru(embedded)  # TODO: add state tuple

        if pack_padded:
            output, output_lens = pad_packed_sequence(output, batch_first=True)

        # Sum outputs of bidirectional RNN:
        # [B, T, 2*H] > [B, T, H]
        if self.bidirectional_encoder:
            output = output[:, :, :self.hidden_dim] + output[:, :, self.hidden_dim:]

        return output, hidden

 

In [101]:
class Decoder(nn.Module):
    def __init__(self, config):
        super(Decoder, self).__init__()
        self.source_dim = config["source_dim"]  # vocab size/number of classes
        self.embedding_dim = config["embedding_dim"]  # Can be different from hidden!?
        self.bidirectional_encoder = config.get("bidirectional_encoder", False)
        self.hidden_dim = config["hidden_dim"]
        # self.hidden_dim = 2 * config["hidden_dim"] if self.bidirectional_encoder else config["hidden_dim"]

        # Layers
        self.embedding = nn.Embedding(self.source_dim, self.embedding_dim)
        self.dropout = nn.Dropout(config.get("dropout", 0.0))
        self.gru = nn.GRU(self.embedding_dim + self.hidden_dim, self.hidden_dim, dropout=config.get("dropout", 0.0),
                          bias=True, batch_first=True)
        self.fc = nn.Linear(self.hidden_dim, self.source_dim)  # input classes = output classes
        self.softmax = nn.Softmax(dim=1)

        # For attention
        self.source_layer = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.state_layer = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.weight_layer = nn.Linear(self.hidden_dim, 1)

    def forward(self, input, hidden, enc_outputs):
        """
        Process batch inputs at one time step and decode prediction.

        :param input: [B]
        :param hidden: [1, B, H]
        :param context_vec: [1, B, H]
        :return: prediction (B, H), hidden (1, B, H)
        """

        # print("decoder forward")
        # print(input)

        # Process the batch inputs only at this timestep t
        # Input of shape [B], but we want [B, 1]
        input = input.unsqueeze(1)

        # print(f"input of shape {input.size()}")
        # print(f"context vec of shape {context_vec.size()}")

        # Embed input [B, 1] > [B, 1, E]
        # Note embedding dimension needs to equal hidden if concatenated
        # But could map to same space
        input = self.embedding(input)
        # print(input)

        # Apply attention and get next input
        # [B, 1, 2*H]
        next_input = self.attention(input, hidden, enc_outputs)

        # Sum context vec and input ???padded = field.pad(minibatch)
        # embedded = embedded + context_vec

        # Apply dropout
        # embedded = self.dropout(embedded)

        # embedded = pack_padded_sequence(embedded, input_lens, batch_first=True, enforce_sorted=False)

        # Feed to RNN
        output, hidden = self.gru(next_input, hidden)

        # print(f"output==hidden {output[0][0]==hidden[0][0]}")

        # output, output_lens = pad_packed_sequence(output, batch_first=True)

        # Shape of output: (B, 1, H), we want (B, H)
        output = output.squeeze(1)

        # Get distribution of classes
        output = self.fc(output)

        return output, hidden

    def attention(self, input, hidden, enc_outputs):
        # Calculate attention

        # Calculate attention distribution alpha for all encoder outputs

        # # Find attention using dot product score (source X hidden_target)
        # Didn't work too well
        # # Reshape target hidden: [1, B, H] > [B, H, 1]
        # h_t = hidden.view(-1, self.hidden_dim, 1)
        # # print(f" s_t: {s_t.size()}")

        # # For batch, perform matrix mult of outputs hidden states and hidden state (s_t) at each time step
        # # [B, T, H] X [B, H, 1] = [B, T, 1]
        # # print(f" outputs: {outputs.size()}")
        # score = enc_outputs.bmm(h_t)

        # # Reshape [B, T, 1] > [B, 1, T] and take softmax
        # alphas = self.softmax(score.transpose(1,2))
        # # print(f" alphas: {alphas.size()}")

        # # Find attention. Take weighted sum of hidden states (i.e. linear combo)
        # # [B, 1, T] X [B, T, H] = [B, 1, H]
        # context_vec = alphas.bmm(enc_outputs)

        # Concatenate next input and context
        # [B, 1, H] ; [B, 1, H] = [B, 1, 2*H]
        # print(f" context: {context_vec.size()}")
        # print(f" in: {input.size()}")

        # Find attention using Bahdenah
        # tanh(Vh_s + Wh_t + b)

        # Project source layer: Vh_s
        # [B, T, H] > [B, T, H]
        source_layer = self.source_layer(enc_outputs)

        # Reshape target hidden: [1, B, H] > [B, 1, H]
        h_t = hidden.view(-1, 1, self.hidden_dim)

        # Project target hidden state to layer
        # [B, 1, H] > [B, 1, H]
        state_layer = self.state_layer(h_t)

        # tanh(Vh_s + Wh_t + b)
        # [B, T, H] > [B, T, H]
        score = torch.tanh(source_layer + state_layer)

        # Project to weight layer with 1 unit
        # [B, T, H] > [B, T, 1]
        weights = self.weight_layer(score)

        # print(weights[0])

        weights = self.softmax(weights)

        # print(weights[0])
        # print(weights.size())
        # print(torch.sum(weights, dim=1)[0])

        # Find attention. Take weighted sum of hidden states (i.e. linear combo)
        # [B, 1, T] X [B, T, H] = [B, 1, H]
        context_vec = weights.view(weights.size(0), 1, -1).bmm(enc_outputs)

        # print(context_vec.size())
        next_input = torch.cat([input, context_vec], dim=-1)

        return next_input
        # Sum hidden and context_vec
        # Didn't seem to work to well
        # hidden = hidden + context_vec.view(1, bs, -1)

        # Reshape [B, 1, H] > [1, B, H]
        # hidden = context_vec.view(1, bs, -1)

In [102]:
class Seq2Seq(nn.Module):

    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()

        self.bidirectional_encoder = config.get("bidirectional_encoder", False)
        self.softmax = nn.Softmax(dim=-1)

        # Block
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target=None, source_lens=None, pack_padded=False, teacher_force_ratio=.5, predict=False):
        """
        :param source: [B, T]
        :param target: [B, T]
        :param source_lengths: [T]
        :param pack_padded: bool
        :return: output (B, V), context (B, H), prev_hidden (B, H), weights (B, T)
        """

        target_vocab_size = self.encoder.source_dim  # here same, but may change for MT
        hidden_size = self.decoder.hidden_dim  # Pain in ass 2 * encoder hidden size

        # Encode, only need output for attention
        # out: [B, T, H], for bidirectional torch automatically sums outputs
        enc_outputs, hidden = self.encoder(source, source_lens, pack_padded)

        # Concatenate hidden states for bidirection encoder
        # [2, B, H] > [1, B, 2*H]
        # if self.bidirectional_encoder:
        #   hidden = torch.cat((hidden[0], hidden[1]), dim=-1)
        #   # Add first dim
        #   hidden = hidden.unsqueeze(0)

        if self.bidirectional_encoder:
            hidden = hidden[0] + hidden[1]
            # Add first dim
            hidden = hidden.unsqueeze(0)

        # Iterate through outputs and decode
        bs = source.size(0)

        # Train
        if not predict:


            # Get start token for batch, e.g. [SOS ...]
            target_length = target.size(1)  # includes padding (i.e max length) -- to be ignored by loss func

            # [B, 1]
            # print(f" target: {target.size()}")
            input = target[:, 0]  # SOS
            # print(f" input: {input.size()}")

            # Get all output predictions step by step and store in matrix of shape [B, T, H]
            predictions = torch.zeros(bs, target_length, target_vocab_size).to(device)
            for t in range(target_length):
                # Decode
                output, hidden = self.decoder(input, hidden, enc_outputs)

                # Get predicted output sequences for timestep
                prediction = self.softmax(output)

                # print("prediction", prediction)

                # [B, H] as softmax distribution
                predictions[:, t] = prediction
                prediction = prediction.argmax(1)

                # print(f"Predicted sequence {prediction}; Target {target[:, t]}")

                # Get next input with or without teacher forcing (correction)
                input = target[:, t] if random.random() < teacher_force_ratio else prediction

                # # Decode
                # prediction, hidden = self.decoder(input, hidden, context_vec)

                # # print("prediction", prediction)
                # predictions[:, t] = prediction
                # prediction = prediction.argmax(1)
                # # print(f"Predicted sequence {prediction}; Target {target[:, t]}")
                # input = target[:, t] if random.random() < teacher_force_ratio else prediction

        # Predict
        else:
            input = torch.ones((1, 1))  # SOS
            prediction = torch.zeros((1, 1))
            predictions = []
            while prediction.item() != 2:  # EOS
                prediction, hidden = self.decoder(input, hidden)
                predictions[:, t] = prediction
                prediction = prediction.argmax(1)
                input = prediction
                predictions.append(prediction.item())

        return predictions





In [114]:
def train(model, iterator, criterion, optimizer, epoch, loss_fun=None, writer=None):
    
    model.train()
    train_loss = 0

    for i, batch in enumerate(train_iter):

        # Zero so gradients don't accumate
        optimizer.zero_grad()

        # Move tensors to cuda
        source, source_lens = batch.source
        target, target_lens = batch.target
        source = source.to(device)  # [B,T]
        target = target.to(device)  # [B,T]

        # Encode and decode sequence
        predictions = model(source, target, source_lens, pack_padded=True)

        
        assert len(predictions) == len(target)  

        #         assert len(iterator) == len(source) == len(target)
        #         n = len(source)
        
        if loss_fun is not None:
            calc_loss = loss_fun(predictions, target, target_lens)
            #               print("Calculated loss", calc_loss.item())
            #             print("Averaged loss", calc_loss.item() / n)
            loss = calc_loss




    
        # print(predictions)
        # print(target)

        # Truncate first SOS prediction and reshape [B, T, H] > [B*T, H]
        predictions = predictions[:, 1:, :].reshape(-1, predictions.size(2))
        target = target[:, 1:].reshape(-1)
        
        #         print("Torch CE loss", criterion(predictions, target).item())



        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        
        if i % 1000 == 499:
            print(f"Training loss for epoch\t {epoch}: {train_loss}")
            print(f"Averaged over 1000 epochs\t {train_loss / 500}")
            print(f"Total steps\t {epoch * len(iterator) + i} steps")
            if writer:
                writer.add_scalar("train_loss", train_loss / 500,
                                 epoch * len(iterator) + i)

    return train_loss

In [115]:
def eval(model, iterator, criterion, epoch, loss_fun=None):
    
    model.eval()

    with torch.no_grad():
        
        epoch_loss = 0
        for i, batch in enumerate(iterator):
            # Move tensors to cuda
            source, source_lens = batch.source
            target, target_lens = batch.target
            source = source.to(device)
            target = target.to(device)

            predictions = model(source, target, source_lens, pack_padded=True)


            if loss_fun is not None:
                calc_loss = loss_fun(predictions, target, target_lens)
                #                 print("Calculated loss", calc_loss.item())
                #             print("Averaged loss", calc_loss.item() / n)
                loss = calc_loss
                    
            # Truncate first SOS prediction
            predictions = predictions[:, 1:, :].reshape(-1, predictions.size(2)) 
            target = target[:, 1:].reshape(-1)
            
            assert len(predictions) == len(target)  # B * (T - 1)
            
            # print(f"Predictions of shape {predictions.size()}", predictions)
            # print(f"Target of shape {target.size()}", target)

            #             print("Torch ce loss", criterion(predictions, target).item())

            epoch_loss += loss.item()

            if i % 1000 == 999:
                print(f"Validation loss for epoch\t {epoch}: {train_loss}")
                print(f"Averaged over 1000 epochs\t {dev_loss / 1000}")
                print(f"Total steps\t {epoch * len(iterator) + i} steps")

        return epoch_loss


In [116]:
import random


def random_eval(model, data, r=2):
    model.eval()

    with torch.no_grad():

        for _ in range(r):

            i = random.randrange(0, len(data))
            print(f"random num: {i}")
            source, target = data[i].source, data[i].target

            print(">", " ".join(source))
            print("=", " ".join(target))

            # Perform preprocessing (indexing, etc.)
            source, source_lens = TEXT.process([source])
            target, target_lens = TEXT.process([target])
            #             print(source)
            #             print(target)
            #             print(source_lens)
            #             print(target_lens)

            # Move tensors to cuda
            source = source.to(device)
            target = target.to(device)

            predictions = model(source, target, source_lens, pack_padded=True)

            # Truncate first SOS prediction
            predictions = predictions[:, 1:, :]
            target = target[:, 1:]

            # Get predicted sequences [B, T]
            predicted_seqs = torch.argmax(predictions, -1)

            for i in range(len(predicted_seqs)):
                # for t in range(len(predicted_seqs[i])):
                print(f"  {' '.join([TEXT.vocab.itos[t] for t in predicted_seqs[i, :-1]])}")

                #             print(f"Predictions of shape {predictions.size()}", predictions)

                #             print(f"Predicted seq of shape {predicted_seqs.size()}", predicted_seqs)

                #             print(f"Target of shape {target.size()}", target)


## Retrain, set new params

In [117]:
# Instantiate model


config = dict(
    source_dim=len(TEXT.vocab),
    embedding_dim=512,
    hidden_dim=512,
    layers=1,
    dropout=.5,
    bidirectional_encoder=True,
    bs=32,
    epochs=25,
    lr=.001
)
print(f"Config: {config}")

Config: {'source_dim': 10004, 'embedding_dim': 512, 'hidden_dim': 512, 'layers': 1, 'dropout': 0.5, 'bidirectional_encoder': True, 'bs': 32, 'epochs': 25, 'lr': 0.001}


In [118]:
from torchtext.data import BucketIterator

train_iter, val_iter, test_iter = BucketIterator.splits((train_data, val_data, test_data),
                                                        batch_sizes=(
                                                            config.get("bs"), config.get("bs"), config.get("bs")),
                                                        sort_within_batch=True,
                                                        sort_key=lambda x: len(x.source),
                                                        device="cpu")

In [119]:
batch = next(iter(train_iter))
batch.source[0][0]

tensor([   2,   13,   37,   15,  154,   60,  598,   14,   26,   44,   88,   99,
          42,    4,    5,    0,   47,   15, 1261,  182,   15,   39,   43,    5,
           3])

In [120]:

list(train_iter)[-1].source[0][0]

tensor([   2,   18,  135,   38, 4131,    0,   14,   72,   49,    0,    0,   20,
         309, 3252,    8,    7,   27,   87,   44,   19,    8,    0,    5,  261,
           3])

In [121]:

TEXT.vocab.itos[25]

'these'

In [122]:
list(train_iter)[-1].target[0][0]  # Batch > sents, lengths

tensor([  2, 144,   3,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1])

In [123]:
encoder = Encoder(config).to(device)
decoder = Decoder(config).to(device)
model = Seq2Seq(encoder, decoder).to(device)
# model = torch.load(path)

# for n, p in model.named_parameters():
#     if p.requires_grad:
#         print(n, p, p.size())

** Training
loop **

In [124]:
from torch.utils.tensorboard import SummaryWriter

# path = 
writer = SummaryWriter("runs/half")

criterion = nn.CrossEntropyLoss(ignore_index=1)
optimizer = optim.Adam(model.parameters())


def loss_fun(predictions, target, target_lens, reduction=torch.sum):
    """

        :param predictions: [B, T, C]
        :param target: [B, T]
        :param target_lens: [B]

        :return: loss: [B]

    """
    assert len(predictions) == len(target) == len(target_lens)
    
    # Create mask
    bs, max_len = target.size()
    mask = torch.zeros(bs, max_len)
    for i in range(bs):
        #         print(i, target_lens[i].item(), max_len)
        #         print(mask)
        mask[i, :target_lens[i].item()] = 1
    
    # Remove SOS and reshape
    predictions = predictions[:, 1:, :].reshape(-1, predictions.size(2)) # [B*(T-1), C]
    target = target[:, 1:].reshape(-1) # [B*(T-1)]
    mask = mask[:, 1:].reshape(-1)  # [B*(T-1)]
    
    # Count total sequences (excluding masked positions)
    assert len(predictions) == len(target) == len(mask)
    #     n = len(predictions)
    n = sum(mask).item()
    
    prob_correct_seqs = torch.gather(predictions, dim=1, index=target.unsqueeze(1)).cpu()
    # print(prob_correct_seqs)
    
    # Calculate NLL (do before mask to avoid inf if zero used for mask)
    prob_correct_seqs = prob_correct_seqs
    nll = -torch.log(prob_correct_seqs) 
    #     print(nll)
    
    # Reduce nll loss across batch filtered with mask
    loss = reduction(nll * mask)
    #     print(f"Loss before: {loss}; and averaged loss over {n} samples: {loss / n}")
    
    return loss / n
    

for epoch in tqdm(range(1, (config.get("epochs") + 1))):
    train_loss = train(model, train_iter, criterion, optimizer, epoch, loss_fun, writer)
    dev_loss = eval(model, val_iter, criterion, epoch, loss_fun)
    
    print(f"\nTotal training loss for epoch {epoch}:\t {train_loss}")
    print(f"\nTotal validation loss for epoch {epoch}:\t {dev_loss}")

    print("Random prediction from training data")
    random_eval(model, train_data, 2)
    print("Random prediction from validation data")
    random_eval(model, val_data, 2)

    path = "models/model-half.pt"
    torch.save(model, path)


  0%|          | 0/25 [00:00<?, ?it/s]

Training loss for epoch	 1: 462304.74395751953
Averaged over 1000 epochs	 924.609487915039
Total steps	 2933 steps
Training loss for epoch	 1: 1282471.683959961
Averaged over 1000 epochs	 2564.943367919922
Total steps	 3933 steps


  4%|▍         | 1/25 [05:30<2:12:13, 330.57s/it]


Total training loss for epoch 1:	 2022114.1791687012

Total validation loss for epoch 1:	 206980.54913330078
Random prediction from training data
random num: 41732
> excellent product . anytime i 'm feeling icky , i mix a packet with water and drink - up . i know i 'll be feeling much better soon .
= used this product for years
  excellent <EOS> <EOS> <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
random num: 18724
> the coffee is wonderful with no bitter taste or after taste . i prefer a cappucino style of coffee with all ingredients pre mixed . this is one of the best that i have tried and i know i will be re - ordering when i am done with my original purchase .
= grove square french vanilla cappuccino
  great coffee coffee vanilla <EOS> <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Random prediction from validation data
random num: 6511
> we have be

  8%|▊         | 2/25 [11:00<2:06:35, 330.26s/it]

...

 92%|█████████▏| 23/25 [2:06:06<10:57, 328.79s/it]

Training loss for epoch	 24: 266425.39471435547
Averaged over 1000 epochs	 532.850789428711
Total steps	 58915 steps
Training loss for epoch	 24: 813502.6412963867
Averaged over 1000 epochs	 1627.0052825927735
Total steps	 59915 steps

Total training loss for epoch 24:	 1339692.3770446777

Total validation loss for epoch 24:	 215497.41638183594
Random prediction from training data
random num: 65805
> nice mix .   but would only be truly happy if i could pick out which cups are included in the package .
= like coffee
  <unk> a <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
random num: 12709
> we really enjoy this rice and purchase it whenever and wherever we can find it . it 's worth the effort for the nice nutty flavor is an enjoyable change from our normal basmati .
= great flavor and texture
  great rice <EOS> texture <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

 96%|█████████▌| 24/25 [2:11:35<05:28, 328.75s/it]

Training loss for epoch	 25: 260733.88711547852
Averaged over 1000 epochs	 521.467774230957
Total steps	 61349 steps
Training loss for epoch	 25: 813175.8175354004
Averaged over 1000 epochs	 1626.3516350708007
Total steps	 62349 steps

Total training loss for epoch 25:	 1342915.4247131348

Total validation loss for epoch 25:	 216692.54846191406
Random prediction from training data
random num: 65709
> you really ca n't go wrong with any of theo 's chocolates , but i 'm partial to this one - i love dark chocolate , and the texture that the almonds add is lovely . to my palate , this is much smoother than many of the other 70%+ dark chocolates .
= theo makes amazing chocoloates
  oh <EOS> <pad> ! <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
random num: 24913
> i purchased due to one of the review saying this was much better than yoki tea , i feel this tea has a bitter taste and not much ginger , in fact i added ginger , 

100%|██████████| 25/25 [2:17:03<00:00, 328.96s/it]
